In [ ]:
import asyncio

keeping={}

class ClientServerProtocol(asyncio.Protocol):
    
    def connection_made(self, transport):
        self.transport = transport

    def data_received(self, data):
        self.transport.write(self.process_data(data.decode('utf-8').strip('\r\n')).encode('utf-8'))
        
       
    def process_data(self, cmd):
        data = cmd.split(' ')
        if data[0] == "get":
             return self.data_get(data[1])
        elif data[0] == "put":
            return self.data_put(data[1], data[2], data[3])
        else:
            return 'error\nwrong command\n\n'
        
        
    def data_put(self,key, value, date):
        
        if not key in keeping:
            keeping[key] = []
            
        if not (date, value) in keeping[key]:
            keeping[key].append((date, value))
            keeping[key].sort(key=lambda x: x[0])
            
        return 'ok\n\n'
        

    def data_get(self, key):
        
        d='ok\n'
        
        if key == '*':
            for k, v in keeping.items():
                for x in v:
                    d=f'{d}{k} {x[1]} {x[0]}\n'
        else:
            if key in keeping:
                for x in keeping[key]:
                    d=f'{d}{key} {x[1]} {x[0]}\n'
        
        return d + '\n'
        

In [ ]:
def run_server(host='127.0.0.1', port=12000):
    loop = asyncio.get_event_loop()
    coro = loop.create_server(ClientServerProtocol, host, port)
    server = loop.run_until_complete(coro)

    try:
        loop.run_forever()
    except KeyboardInterrupt:
        pass

    server.close()
    loop.run_until_complete(server.wait_closed())
    loop.close()